Projet Optimisation Groupe 2 : STOCK 
DESJONQUERES Henri, MARZIN Ewen 

I. Etude du problème d'optimisation

1. 

La partie $  c^{T} \cdot r =  \displaystyle \sum_{j=1}^{n} c_j * r_j$ correspond aux dépenses totales d'achat des matières premières. 

$\min \{ q, d \}$ correspond au nombre de produit vendu qui est en fait bien le minimum entre le nombre de produit fabriqués et la quantité prévisionnelle de vente : en effet si on a fabriqué trop peu de produits par rapport à la prévision de ventes, on ne pourra vendre qu'au plus tous les produits fabriqués et inversemment si on a fabriqué plus de produit que la quantité prévisionnelle des ventes alors on vend un nombre de produits qui correpond à cette prévision de ventes. 

La partie $ v^{T} \cdot \min (q, d) = \displaystyle \sum_{j=1}^{n} v_j * \min (q_j, d_j)$ correspond alors au total de recettes des ventes des ventes des produits. 
La différence de la seconde moins la première nous donne donc le gain final de la boulangerie (recettes - dépenses)

2. 

La fonction minimum est une fonction qui n'est pas différentiable donc cela complexifie la résolution du problème d'optimisation 

3. 

On considère la fonction $h : (d,q) \rightarrow h(d,q)$ définie par pour tout $i \in \llbracket ~1;p~ \rrbracket$ : $h_i (d,q) = \frac{q_i \cdot e^{-\alpha \cdot q_i} + d_i \cdot e^{-\alpha \cdot d_i}}{e^{-\alpha \cdot q_i} + e^{-\alpha \cdot d_i} }$ , avec $\alpha >> 1$. 

Comme $\alpha >> 1$, si $q_i > d_i$ alors on a $ e^{-\alpha \cdot d_i} >> e^{-\alpha \cdot q_i}$ (car $e^{\alpha \cdot (q_i - d_i)}  \rightarrow + \infty$ lorsque $\alpha \rightarrow +\infty$)

Ainsi on a dans ce cas : $h_i(d,q) = \frac{q_i \cdot e^{- \alpha \cdot (q_i - d_i)} + d_i}{e^{- \alpha \cdot (q_i - d_i)} + 1 }$, et avec $e^{- \alpha \cdot (q_i - d_i)} << 1$ on obtient que : 
$h_i(d,q) = \frac{d_i + o(1)}{1+o(1)} ≈ d_i = \min(d_i,q_i)$

Si $q_i < d_i$ un raisonnement similaire nous donne dans ce cas : 
$h_i(d,q) = \frac{d_i + o(1)}{1+o(1)} ≈ q_i = \min(d_i,q_i)$

Et si $d_i = q_i$, on obtient directement $h_i(d,q) = q_i = d_i = \min(q_i, d_i)$

On a donc bien une bonne approximation de la fonction min avec la fonction h. 
Il est plus intéressant de considérer ce problème là car la fonction h est $C^1$ comme composée de fonctions $C^1$ 

4. 

En posant $z = (q, r)$, les variables de décision, soit $z = (z_1, z_2)$, avec $z_1 = q \in \mathbb R^p_+$ et $z_2 = r \in \mathbb R^m_+$, on a alors $n = p + m$ variables de décisions.

Les contraintes s'écrivent alors : 
$c(z) = (- z_1, - z_2, Az_1 - z_2) \leq 0$

Et on cherche à résoudre le problème suivant, avec $f(z) = c^T \cdot z_2 - v^T \cdot h(z_1,d)$ la fontion objectif à minimiser (pour maximiser le profit = -$f(z)$, on minimise f)

$$ \min_{c(z) \leq 0} f(z) $$

II. Etude et résolution numérique 

5. 

Comme on est dans le cadre d'un problème d'optimisation sous contraintes linéaires, on aurait pu penser à la méthode du simplexe, mais celle-ci necessite une égalité dans les contraintes à la place d'une inégalité. 

On peut donc utiliser une méthode SLSQP pour résoudre le problème numériquement

II. 6. 

In [60]:
import numpy as np
from scipy.optimize import minimize


# Données
alpha = 0.1
c = 1e-3 * np.array([30, 1, 1.3, 4, 1])
v = np.array([0.9, 1.5, 1.1])
d = np.array([400, 67, 33])
A = np.array([
    [3.5, 2, 1],
    [250, 80, 25],
    [0, 8, 3],
    [0, 40, 10],
    [0, 8.5, 0]
])
m, p = A.shape

# Fonction h 
def h(q, d, alpha):
    return (q * np.exp(-alpha * q) + d * np.exp(-alpha * d)) / (np.exp(-alpha * q) + np.exp(-alpha * d))


def fonction(z):
    q = z[:p]
    r = z[p:]
    revenue = v @ h(q, d, alpha)
    cout = c @ r
    return - (revenue - cout)


def contraintes(z):
    q = z[:p]
    r = z[p:]
    return r - A @ q 

cons = {'type': 'ineq', 'fun': contraintes}

z0 = np.zeros(p + m)


res = minimize(fonction, z0, method='SLSQP', constraints=cons)


q_opt = res.x[:p]
r_opt = res.x[p:]

print("q* (quantités produites) =", q_opt)
print("r* (matières premières achetées) =",r_opt)

q* (quantités produites) = [402.10730848  73.1035627   42.43687785]
r* (matières premières achetées) = [  1596.01958291 107436.03408132    712.13913511   3348.51128631
    621.38028291]


6. 
On obtient des quantités proches de la demande pour alpha petit (de l'ordre de 0.1) et des quantités nettement plus faibles (autour de 73 pour la premiere valeur par rapport à 400) quand alpha est plus grand (10). 
Cela semble étonnant car on s'attend a avoir une production assez proche de la demande au minimum pour maximiser le profit et la fonction h est censée approchée la fonction min du problème quand alpha devient grand devant 1. 

On trouve donc l'inverse et des quantités qui s'éloignent de la demande à mesure que alpha grandit et donc que la fonction h approche la fonction min du problème.

II. 7. (a) On souhaite cette fois-ci maximiser l'espérance du profit qui vaut : 

$\mathbb E (v^T \cdot h(q, d^k) -c^T\cdot r) = \displaystyle \sum_{k=0}^{K} \pi^k \cdot (v^T \cdot h(q,d^k) - c^T \cdot r)$. 

Les variables de décisions : $z = (q, r)$ restent identiques tout comme les contraintes $c(z) = (- z_1, - z_2, Az_1 - z_2) \leq 0$ et la fonction objectif à minimiser est alors : 

$f(z) = - \displaystyle \sum_{k=0}^{K} \pi^k \cdot (v^T \cdot h(q,d^k) - c^T \cdot r)$

(b)

In [61]:
d1 = np.array([400, 67, 33])
d2 = np.array([500, 80, 53])
d3 = np.array([300, 60, 43])
demands = [d1, d2, d3]
probs = [0.5, 0.3, 0.2]

def fonction2(z):
    q = z[:p]
    r = z[p:]
    revenu_moyen = sum(p * (v @ h(q, d, alpha)) for p, d in zip(probs, demands))
    cout = c @ r
    return - (revenu_moyen - cout)

res = minimize(fonction2, z0, method='SLSQP', constraints=cons)


q_opt = res.x[:p]
r_opt = res.x[p:]

print("Quantités optimales avec incertitude:", q_opt)
print("Matières premières achetées:",r_opt)

Quantités optimales avec incertitude: [406.66848798  77.13245587  54.30512469]
Matières premières achetées: [  1631.90974433 109195.34658031    779.97502099   3628.34948152
    655.62587486]


On obtient là encore des résulats cohérents proches de la demande (et de la demande pondérée) lorsque alpha est faible, de l'ordre de 0.1 et à l'inverse quand alpha est grand (alpha = 10), les valeurs renvoyées sont assez significativement inférieures à celles de la demande, surtout pour la première valeur. 

Les réponses sont en adéquation avec celles de la question 6., les légères différences s'expliquant par la considération des 3 demandes différentes, que l'on a pondérées.

III. Etude du problème non régularisé 

8. (a) On a toujours $z = (z_1, z_2) = (q, r) \in \mathbb R^p x R^m$ et $n = p+m$ le nombre de ces variables de décisions. 

On a toujours les contraintes $c(z)$ telles que : 
$c(z) = (- z_1, - z_2, Az_1 - z_2) \leq 0$

Néanmoins, la fonction à minimiser s'écrit cette fois sous la forme :
$f(z) = c^T \cdot z_2 - v^T \cdot min\{ z_1, d \}$

(b) Supposons par l'absurde que la contrainte (1) : $Az_1 - z_2 = Aq - r \leq 0$, n'est pas active à l'optimum. On note $z^* = (q^*, r^*)$ le minimiseur de f.

On a alors l'existence d'un indice $i_0 \in \llbracket 1 ~;~ m \rrbracket$ tel que $(Aq^*)_{i_0} < r^*_{i_0}$. 

On pose $\~{r} = (r^*_1, ..., r^*_{i_0 -1}, (Aq^*)_{i_0}, r^*_{i_0 +1}, ..., r^*_m)$, on a alors $Aq^* \leq \~{r}~$ et comme $(Aq^*)_{i_0} < r^*_{i_0}$ : 

$v^T \cdot min\{q^*, d\} - c^T \cdot \~{r} > v^T \cdot min\{q^*, d\} - c^T \cdot r*$

Soit $Aq^* \leq \~{r}$, $q^* \geq 0$, $\~{r} \geq 0$ et $f(q^*, \~{r}) < f(q^*, r^*)$. 

CONTRADICTION avec l'hypothèse de $(q^*,r^*)$ minimiseur de f.

On conlut que la contrainte est donc bien active à l'optimum

(c) On peut donc restreindre l'étude en remplaçant la contrainte d'inégalité par une contrainte d'égalité : 
$Aq = r$, ou $Az_1 = z_2$. 

Comme on a établi une relation entre q et r, on peut remplacer r par la valeur de $Aq$ correspondante et donc se ramener à un problème de minimisation à une seule variable : q. 

Le nombre de variables de décision est donc n = p et l'unique contrainte restante est la positivité de q : $c(q) = -q \leq 0$ (que l'on peut résumer comme $q \geq 0$ )

Le problème de minimisation devient donc, avec $\~{f} (q) = \displaystyle \sum_{k=0}^{K} (c^T \cdot (Aq) - v^T \cdot min\{q, d^k \}) \cdot \pi^k$ la fontion objectif à minimiser :


$$ \min_{q \geq 0} \~{f}(q) $$

9. (a) 
En premier lieu, pour $k \in \llbracket 1~;~ K \rrbracket$ on a que si $u_k \leq d^k$ et $u_k \leq q$, alors $u_k \leq min\{q,d\}$ et alors comme les coefficients de v sont positifs ou nuls :

$v^T \cdot u_k = \displaystyle \sum_{i=1}^{p} v_i \cdot (u_k)_i \leq v^T \cdot min\{q, d^k\}$

Ainsi $$\max_{u_k \leq min\{q, d\}} v^T \cdot u_k \leq \max_{q} v^T \cdot min\{q,d^k\}$$

On prend $q^*$ tel que $$\max_{q} v^T \cdot min\{q,d^k\} = v^T \cdot min\{q^*, d^k \}$$ et $u_k^*$ tel que $$\max_{u_k \leq min\{q^*, d^k\}} v^T \cdot u_k = v^T \cdot u_k^*$$  


Supposons par l'absurde qu'on ait $v^T \cdot u_k^* <  v^T \cdot min\{q^*, d^k \}$. 

Posons $\~{u_k} = \frac{u_k^* + min\{q^*, d^k\}}{2} \geq u_k^*$, et au moins une des coordonnées est strictement supérieure à l'autre (car $v^T \cdot u_k^* <  v^T \cdot min\{q^*, d^k \}$)

On a aussi $\~{u_k} \leq min\{q^*, d^k\}$, on a donc : 

$v^T \cdot u_k^* < v^T \cdot \~{u_k} \leq v^T \cdot min\{q^*, d^k\}$. 

CONTRADICTION avec la propriété de maximum vérifié par $u_k^*$

(b) 

On veut de nouveau maximiser l'espérance. En remplaçant pour $k \in \llbracket 1~;~ K \rrbracket$ : $min\{q^*, d^k\}$ par $u_k$ on a alors avec $z = (u_1,..., u_K, q)$, $n = (K+1) \cdot p$ 

et les contraintes pour $k \in \llbracket 1~;~ K \rrbracket$ : $c_{k1} : u_k - q \leq 0$, $c_{k2} : u_k - d^k \leq 0$ : 

$$\min_{u_k \leq q, u_k \leq d^k, k \in \llbracket 1~;~ K \rrbracket } \displaystyle \sum_{k=0}^{K} (c^T \cdot (Aq) - v^T \cdot u_k) \cdot \pi^k$$

Sans le minimum, la fonction est maintenant différentiable et la résolution du problème est donc facilitée. 

(c)

In [62]:
K = 3
def fonction3(z):
    u = [0 for k in range(K)]
    for k in range(K):
        u[k] = z[k*p : (k+1)*p]
    q = z[K*p :]
    S=0
    for k in range(K):
        S -= (v @ u[k] - c @ (A @ q))*probs[k]
    return S


def contraintes3(z):
    u = [0 for k in range(K)]
    for k in range(K):
        u[k] = z[k*p : (k+1)*p]
    q = z[K*p :]
    c=[]
    for k in range(K):
        c.append(q - u[k])
        c.append(demands[k] - u[k])
    return np.concatenate(c)
    

cons3 = {'type': 'ineq', 'fun': contraintes3}

z0 = np.zeros((K+1)*p)

res = minimize(fonction3, z0, method='SLSQP', constraints=cons3)

q_opt = res.x[K*p:]

print("q* (quantités produites) =", q_opt)


q* (quantités produites) = [400.  80.  53.]


On remarque que les quantités produites obtenues sont proches des demandes, (et de la pondération des demandes) ce qui est cohérent avec notre intuition qu'à l'optimum,  une production proche de la demande permet bien de maximiser le profit. Cela contraste donc avec nos résultats précédents assez éloignés de la demande à l'optimum.

10. 
(a) On souhaite réaliser une méthode de résolution non lisse dans le cadre de la fonction min non différentiable. 
On va utiliser la méthode de minimisation proximale qui utilise l'opérateur proximal de la fonction.

(b)

In [63]:
def f_min(q, d):
    return np.minimum(q, d)


def fonction4(q):
    cout4 = c @ A @ q
    return sum(p * (cout4 - np.dot(v, f_min(q, d))) for p, d in zip(probs, demands))


def proximal(x, l, f):
    def prox_obj(s):
        return f(s) + np.linalg.norm(s - x)**2 / (2 * l)
    res = minimize(prox_obj, np.zeros_like(x), method='SLSQP')
    return res.x

def minimisation_proximal(epsilon,l):
    x = [np.zeros(3),proximal(np.zeros(3),l,fonction4)]
    i = 1
    while np.linalg.norm(x[i] - x[i-1]) >= epsilon:
        x += [proximal(x[i],l,fonction4)]
        i += 1
    return x[-1]

print("Quantités optimales :", minimisation_proximal(0.1, 1))

Quantités optimales : [400.00002905  80.00000539  53.00006873]


On remarque qu'on a une très bonne précision des quantités renvoyées quand epsilon = 0.1, les quantités etant encore proches de la demande, et très proches de celles trouvées à la question 9.c), ce qui semble plus pertinent que les premiers résultats.